In [ ]:
pip install langdetect

     |████████████████████████████████| 983kB 13.4MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=f6f8a8cc2bd0526c55af65dd32da6f6fe37a8862e9953e5b56b82ba1358b76f7
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
Successfully built langdetect


In [ ]:
# Import dependencies
from sklearn.preprocessing import LabelEncoder
from langdetect import detect, detect_langs
from google.colab import files
import pandas as pd
import time

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Shareable link
link = 'https://drive.google.com/file/d/1pFmCSD7c4HEB9IRhGweuIE8SKi42M0lL/view?usp=sharing'
id = '1pFmCSD7c4HEB9IRhGweuIE8SKi42M0lL'


In [ ]:
# Read in DataFrame
downloaded = drive.CreateFile({'id': id}) 
downloaded.GetContentFile('song_lyrics.csv')  
song_lyrics_df = pd.read_csv('song_lyrics.csv')
song_lyrics_df.head()

,song,song_id,artist,artist_id,category,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0,Last night all I think about is you Dont stop...,1
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1


In [ ]:
# Remove the songs without lyrics
song_lyrics_df = song_lyrics_df.dropna()

In [ ]:
# Add encoded category column
le = LabelEncoder()
new_column = le.fit_transform(song_lyrics_df['category']) 
song_lyrics_df.insert(5, 'category_id', new_column)

# Remove non_alpha_words column and null values
#song_lyrics_df = song_lyrics_df.drop('non_alpha_words', axis=1)
#song_lyrics_df = song_lyrics_df.dropna()

# Save DataFrame to CSV
song_lyrics_df.to_csv('encoded_df.csv', index=False)
#files.download('encoded_df.csv')
song_lyrics_df.head()

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.1700,0.8330,0.001790,0.1450,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.2840,0.0805,0.000000,0.3660,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.2130,0.000000,0.1340,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,8,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,-6.900,1.0,0.0944,0.4400,0.000007,0.0921,0.531,80.870,238805.0,4.0,Last night all I think about is you Dont stop...,1
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,-5.905,1.0,0.0373,0.3180,0.000000,0.4140,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1


In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,699 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [870 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  I

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lyrics_NLP').getOrCreate()

In [ ]:
# Read in DataFrame
music_df = spark.read.csv('encoded_df.csv', sep=",", header=True)
music_df.show()


+--------------------+--------------------+--------------+--------------------+--------+-----------+--------------------+---------------------+------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+------------------+------------------+--------------------+-----------------------+--------------------+---------------+
|                song|             song_id|        artist|           artist_id|category|category_id|              genres|audio_ft_danceability|   audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|  audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|
+--------------------+--------------------+--------------+--------------------+--------+-----------+--------------------+---------------------+------------------+

In [ ]:
# Import functions
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [ ]:
# Tokenize lyrics
tokenizer = Tokenizer(inputCol='lyrics', outputCol='words')
tokenized_df = tokenizer.transform(music_df)
tokenized_df.show()

+--------------------+--------------------+--------------+--------------------+--------+-----------+--------------------+---------------------+------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+------------------+------------------+--------------------+-----------------------+--------------------+---------------+--------------------+
|                song|             song_id|        artist|           artist_id|category|category_id|              genres|audio_ft_danceability|   audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|  audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|               words|
+--------------------+--------------------+--------------+--------------------+--------+-----------+--------------------

In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol='words', outputCol='filtered')
removed_df = remover.transform(tokenized_df)
removed_df.show()

+--------------------+--------------------+--------------+--------------------+--------+-----------+--------------------+---------------------+------------------+------------+-------------------+-------------+--------------------+---------------------+-------------------------+-------------------+------------------+------------------+--------------------+-----------------------+--------------------+---------------+--------------------+--------------------+
|                song|             song_id|        artist|           artist_id|category|category_id|              genres|audio_ft_danceability|   audio_ft_energy|audio_ft_key|  audio_ft_loudness|audio_ft_mode|audio_ft_speechiness|audio_ft_acousticness|audio_ft_instrumentalness|  audio_ft_liveness|  audio_ft_valence|    audio_ft_tempo|audio_ft_duration_ms|audio_ft_time_signature|              lyrics|non_alpha_words|               words|            filtered|
+--------------------+--------------------+--------------+--------------------

In [ ]:
# Convert DataFrame to Pandas
nlp_df = removed_df.toPandas()
nlp_df.head()

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou..."
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,8,"['shiver', 'indietronica', 'gauze']",0.7609999999999999,0.525,11.0,-6.9,1.0,0.0944,0.44,6.7e-06,0.0921,0.531,80.87,238805.0,4.0,Last night all I think about is you Dont stop...,1,"[, last, night, all, i, think, about, is, you,...","[, last, night, think, dont, stop, baby, walk,..."
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.6659999999999999,0.593,2.0,-5.905,1.0,0.0373,0.318,0.0,0.414,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1,"[, are, you, okay, ?, cause, youre, the, one, ...","[, okay, ?, cause, youre, one, needed, space, ..."


In [ ]:
# Remove instrumental songs
for index, row in nlp_df.iterrows():
    if row['lyrics'].lower() == 'instrumental':
        nlp_df = nlp_df.drop(index)
nlp_df.head()

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough...","(1, en)"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,...","(1, en)"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou...","(1, en)"
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,8,"['shiver', 'indietronica', 'gauze']",0.7609999999999999,0.525,11.0,-6.9,1.0,0.0944,0.44,6.7e-06,0.0921,0.531,80.87,238805.0,4.0,Last night all I think about is you Dont stop...,1,"[, last, night, all, i, think, about, is, you,...","[, last, night, think, dont, stop, baby, walk,...","(1, en)"
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.6659999999999999,0.593,2.0,-5.905,1.0,0.0373,0.318,0.0,0.414,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1,"[, are, you, okay, ?, cause, youre, the, one, ...","[, okay, ?, cause, youre, one, needed, space, ...","(1, en)"


In [ ]:
# Create a list of languages
t0 = time.time()
languages = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    try:
        langs = detect_langs(' '.join(map(str, filtered_words)))
        languages.append((len(langs), str(langs[0])[0:2]))
    except:
        languages.append(None)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(languages)

Run time: 85.17528176307678 seconds


8611

In [ ]:
# Find songs indeces with one language that is not English
no_lang = []
one_lang_not_english = []
mult_langs_not_english = []
for id, item in enumerate(languages):
  if item == None:
    no_lang.append(id)
  if item != None:
    if (item[0] == 1) & (item[1] != 'en'):
      one_lang_not_english.append(id)
    if (item[0] > 1) & (item[1] != 'en'):
      mult_langs_not_english.append(id)
print(len(no_lang))
print(len(one_lang_not_english))
len(mult_langs_not_english)

9
286


135

In [ ]:
# Print songs with no language
for index in no_lang:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])

                   song artist lyrics
428  How Low Can You Go     LP    4.0
                song   artist lyrics
2245  Salomon's Gate  Beherit      ?
                 song    artist    lyrics
2345  Dialectic Chaos  Megadeth          
             song artist lyrics
3469  Lost on You     LP    4.0
                     song         artist  lyrics
4270  Baby Can I Hold You  Tracy Chapman  74.375
                    song         artist  lyrics
4296  Give Me One Reason  Tracy Chapman  100.22
                                song                  artist lyrics
5751  "Tosca / Act 2: ""Vissi d'arte  7llzoWbL1ToAHq9IL3F84D    3.0
                       song            artist    lyrics
5812  """We're Not Programs   We're People"""  310253.0
                                                 song  ... lyrics
5859  "Sonata No. 14 ""Moonlight"" in C-Sharp Minor""  ...    3.0

[1 rows x 3 columns]


In [ ]:
# Print non English songs with multiple languages
for index in mult_langs_not_english:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])

In [ ]:
# Print non English songs with one language
for index in one_lang_not_english:
  print(nlp_df[index:index+1][['song', 'artist', 'lyrics']])
songs_to_keep = [78, 453, 1084, 1089, 1634, 2151, 2405, 2430, 2529, 2838, 2993, 3633, 3737, 3805, 3846, 4215, 4389, 4664, 5070, 5148,
                 5209, 5265, 5360, 5661, 5732, 5786, 5828, 5958, 6119, 6269, 6464, 6472, 6495, 6502, 6595, 6627, 6696, 6729, 7277]

In [ ]:
# Add a language column to the DataFrame and replace the filtered column
nlp_df['language'] = languages
nlp_df.head()

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough...","(1, en)"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,...","(1, en)"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou...","(1, en)"
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,8,"['shiver', 'indietronica', 'gauze']",0.7609999999999999,0.525,11.0,-6.9,1.0,0.0944,0.44,6.7e-06,0.0921,0.531,80.87,238805.0,4.0,Last night all I think about is you Dont stop...,1,"[, last, night, all, i, think, about, is, you,...","[, last, night, think, dont, stop, baby, walk,...","(1, en)"
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.6659999999999999,0.593,2.0,-5.905,1.0,0.0373,0.318,0.0,0.414,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1,"[, are, you, okay, ?, cause, youre, the, one, ...","[, okay, ?, cause, youre, one, needed, space, ...","(1, en)"


In [ ]:
# Remove songs that do not have English lyrics
for index, row in nlp_df.iterrows():
    if row['language'] != 'en':
        nlp_df = nlp_df.drop(index)
nlp_df.head()

,song,song_id,artist,artist_id,category_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,words,filtered,language
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,32,toplists,"['pop', 'viral', 'canadian', 'post-teen', 'dan...","['pop', 'viral', 'canadian', 'dance']",0.652,0.383,2,-7.0760000000000005,0,0.0516,0.0676,0.0,0.0828,0.5489999999999999,145.765,178994,4,You put me on a pedestal and tell me Im the b...,"[, you, put, me, on, a, pedestal, and, tell, m...","[, put, pedestal, tell, im, best, raise, sky, ...",en
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,32,toplists,"['pop', 'electropop']",['pop'],0.889,0.34,11,-7.773,0,0.0697,0.218,0.13,0.055,0.716,94.009,174321,4,Im not your friend Or anything damn You think...,"[, im, not, your, friend, or, anything, damn, ...","[, im, friend, anything, damn, think, youre, m...",en
2,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,32,toplists,"['pop', 'post-teen']",['pop'],0.736,0.802,0,-4.759,1,0.0864,0.468,0.0,0.094,0.675,144.005,172325,4,Heaven sent you to me Im just hopin I dont re...,"[, heaven, sent, you, to, me, im, just, hopin,...","[, heaven, sent, im, hopin, dont, repeat, hist...",en
3,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,32,toplists,"['lgbtq+', 'pop', 'queer', 'country']","['pop', 'country']",0.81,0.511,5,-6.924,0,0.16399999999999998,0.12,0.0,0.0832,0.8370000000000001,151.947,154998,4,T T Tay Keith Took it to ten Hey Ayy its a ...,"[, t, t, tay, keith, took, it, to, ten, hey, ,...","[, tay, keith, took, ten, hey, , , ayy, holida...",en
4,On Me,4Iedi94TIaB2GGb1nMB68v,Lil Baby,5f7VJjfbwm532GiveGC0ZK,32,toplists,"['atl', 'rap', 'trap']","['atl', 'rap', 'trap']",0.856,0.564,6,-6.295,0,0.392,0.0032700000000000003,0.0,0.134,0.483,77.972,135960,4,Whats happenin Chi Chi ? Fill the bando up ...,"[, whats, happenin, chi, chi, ?, , , fill, the...","[, whats, happenin, chi, chi, ?, , , fill, ban...",en


In [ ]:
# Remove songs with no language
for index, row in nlp_df.iterrows():
    if row['language'] == None:
        nlp_df = nlp_df.drop(index)
nlp_df.head()

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,0.17,0.833,0.00179,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"[, im, like, the, water, when, your, ship, rol...","[, im, like, water, ship, rolled, night, rough...","(1, en)"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,0.284,0.0805,0.0,0.366,0.7559999999999999,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"[, didnt, even, wanna, go, out, whyd, you, cal...","[, didnt, even, wanna, go, whyd, call, ?, ive,...","(1, en)"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,0.0299,0.213,0.0,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"[, you, cut, out, a, piece, of, me, and, now, ...","[, cut, piece, bleed, internally, left, withou...","(1, en)"
3,Heat Waves,3USxtqRwSYz57Ewm6wWRMp,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,pop,8,"['shiver', 'indietronica', 'gauze']",0.7609999999999999,0.525,11.0,-6.9,1.0,0.0944,0.44,6.7e-06,0.0921,0.531,80.87,238805.0,4.0,Last night all I think about is you Dont stop...,1,"[, last, night, all, i, think, about, is, you,...","[, last, night, think, dont, stop, baby, walk,...","(1, en)"
4,r u ok,7rq1zCf90CnU7OalBLIgNp,Tate McRae,45dkTj5sMRSjrmBSBeiHym,pop,8,"['dance', 'pop', 'electropop', 'post-teen']",0.6659999999999999,0.593,2.0,-5.905,1.0,0.0373,0.318,0.0,0.414,0.329,140.013,185850.0,4.0,Are you okay ? Cause youre the one who needed...,1,"[, are, you, okay, ?, cause, youre, the, one, ...","[, okay, ?, cause, youre, one, needed, space, ...","(1, en)"


In [ ]:
# Find the number of songs that do not have English lyrics
len(nlp_df[nlp_df['language'].apply(lambda x: 'en' not in x)])

421

In [ ]:
# Save DataFrame to CSV
nlp_df.to_csv('nlp_df.csv', index=False)
files.download('nlp_df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save DataFrame to CSV file
removed_df.toPandas().to_csv('nlp_df.csv')
#files.download('nlp_df.csv')


In [ ]:
# TF-IDF
hashing = HashingTF(inputCol='filtered', outputCol='hashedValues', numFeatures=pow(2,18))
hashed_df = hashing.transform(removed_df)
idf = IDF(inputCol='hashedValues', outputCol='features')
idfModel = idf.fit(hashed_df)
rescaledData = idfModel.transform(hashed_df)
#rescaledData.select('words', 'features').show()
rescaledData.show()